In [1]:
import argparse
import os
import time

import glog, json

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

import torch
import torch.multiprocessing as mp
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.modeling_attn_mask_utils import \
    _prepare_4d_causal_attention_mask


from operator import attrgetter

from pathlib import Path
import sys
notebook_dir = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

from NWC.models import get_model

W1106 12:30:26.501163 1801269 warnings.py:109] /opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

W1106 12:30:27.951740 1801269 warnings.py:109] /opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(

I1106 12:30:28.310747 1801269 utils.py:148] Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
I1106 

In [2]:
class Config:
    def __init__(self, **entries):
        self.__dict__.update(entries)

# comp_model_path = '../NWC/checkpoint/nwc_ql/block_seq_ql_random_scaler_meta-llama--Meta-Llama-3-8B__col_1024_gaussian_padding.pt/M16/lmbda50_rdloss_ql_size16_encdim512_M16_Q4_R0_m0_batch_size2048_total_iter200000_lr0.0001_seed100/best_loss_model_loss_3.87239_bpp_4.65884_MSE_0.0162_total_iter_95000.pth.tar'
# comp_model_path = '/workspace/Weight_compression/NWC/checkpoint/nwc_scale_cond/block_seq_scale_cond_scaler_meta-llama--Meta-Llama-3-8B__scaleH_sig0.0001_std_rnormed_with_col_std_lidx_row_1024.pt/rdloss_size128_encdim1024_M256_Q0_R0_m0_batch_size2048_total_iter200000_lr0.0001_seed100/lmbda50_/best_loss_model_loss_3.94749_bpp_3.26997_MSE_4.91093_total_iter_192500.pth.tar'
comp_model_path = '/home/jgryu/workspace/weight_compression/NWC/checkpoint/nwc_ql/block_seq_ql_random_scaler_meta-llama--Meta-Llama-3-8B__col_1024_gaussian_padding.pt/M16/lmbda300_rdloss_ql_size16_encdim512_M16_Q4_R0_m0_batch_size2048_total_iter200000_lr0.0001_seed100/best_loss_model_loss_5.32101_bpp_5.72603_MSE_0.00289_total_iter_95000.pth.tar'
config = os.path.join(os.path.dirname(comp_model_path), 'config.json')
with open(config, 'r', encoding='utf-8') as file:
    config = json.load(file)
config = Config(**config)

shift, scale = None, None
if config.architecture == 'nwc_ql' and not hasattr(config, "Q"):
    config.Q = 4
if not hasattr(config, "no_layernorm"):
    config.no_layernorm = False


comp_model = get_model(config.architecture, config, scale=scale, shift=shift)
comp_model.config = config
ckpt = torch.load(comp_model_path, weights_only=False)
scale, shift  = torch.zeros(1), torch.zeros(1)

comp_model.load_state_dict(ckpt["state_dict"], strict = False)
comp_model.scale = scale
comp_model.shift = shift
comp_model.eval()
comp_model.update()

comp_model.update(force=True)              # CompressAI: CDF 고정 및 버퍼 등록
comp_model.entropy_bottleneck._quantized_cdf  # 캐시되어 이후 재계산 안 됨

tensor([[    0,     1,     2,  ...,     0,     0,     0],
        [    0,     1,     2,  ...,     0,     0,     0],
        [    0,     1,     2,  ..., 65535, 65536,     0],
        ...,
        [    0,     1,     2,  ..., 65535, 65536,     0],
        [    0,     1,     2,  ...,     0,     0,     0],
        [    0,     1,     2,  ...,     0,     0,     0]], dtype=torch.int32)

In [3]:
import torch
from torch.nn.utils.rnn import pad_sequence

from transformers import LlamaForCausalLM

path = '/home/jgryu/workspace/weight_compression/hf_model_comp/comp_qtip/ckpt/meta-llama--Meta-Llama-3-8B/ql_ldlq128_rnorm_codes_test/lmbda300'
codes_dict = {}
metadata_dict = {}

# n_parm = 0
# n_bit = 0

import time
from collections import defaultdict

device = torch.device('cuda:1')

all_timings = defaultdict(list)

comp_model.to(device)


for i in range(32):
    for m in ['q', 'k', 'v', 'o', 'up', 'down', 'gate']:
        pt = f'{path}/{i}_{m}.pt'
        s = torch.load(pt, weights_only=False, map_location='cpu')

        codes = s['codes']
        tensor_code_list = []
        for c in codes:
            c['q_level'] = c['q_level'].to(device)
            
            out_dec, timings = comp_model.fast_decompress_v2(c)

            for step, duration in timings.items():
                all_timings[step].append(duration)
            
            
total_timings = {}
for step, duration_list in all_timings.items():
    average_duration = sum(duration_list)
    total_timings[step] = average_duration

sorted_total_timings = total_timings.items()

for step, avg_duration in sorted_total_timings:
    print(f"{step:<25}: {avg_duration:.4f} ms")
    
print("\n--- 🚀 GPU 가속 시나리오 계산 (32x 병렬화) ---")
try:
    original_total_time = total_timings['total_decompress_ms']
    overhead_elimination_time = sum(
        value for key, value in total_timings.items() if 'transfer:' in key
    )
    
    original_decode_time = total_timings.get('entropy_dec_loop_decode_call_ms', 0)
    parallelization_factor = 32
    new_decode_time = original_decode_time / parallelization_factor
    time_saved_from_parallelization = original_decode_time - new_decode_time
    total_time_saved = overhead_elimination_time + time_saved_from_parallelization
    new_total_decompress_time = original_total_time - total_time_saved
    percentage_saved = (total_time_saved / original_total_time) * 100
    print(f"원본 총 시간: {original_total_time:.4f} ms")
    print("\n[절감 내역]")
    print(f"  데이터 전송 오버헤드 제거: {overhead_elimination_time:.4f} ms")
    print(f"  CPU 디코딩 시간 (원본): {original_decode_time:.4f} ms")
    print(f"  GPU 디코딩 시간 (32x 병렬): {new_decode_time:.4f} ms")
    print(f"  -> 디코딩 병렬화로 인한 절감: {time_saved_from_parallelization:.4f} ms")
    print("-------------------------------------------------")
    print(f"총 절감 시간 (오버헤드 + 병렬화): {total_time_saved:.4f} ms")
    print(f"✅ 최종 예상 시간: {new_total_decompress_time:.4f} ms")
    print(f"✅ 시간 절감률: {percentage_saved:.2f} %")

except KeyError as e:
    print(f"\n[오류] 계산에 필요한 타이밍 키({e})를 찾을 수 없습니다.")
except Exception as e:
    print(f"\n[오류] 계산 중 예외 발생: {e}")      
          

setup_ms                 : 1053.9334 ms
transfer:indexes         : 22798.9820 ms
transfer:cdf_length,offeset: 620.0715 ms
transfer:cdf             : 949.6154 ms
transfer:entropy_dec_loop_indexes_to_list_ms: 155777.9585 ms
entropy_dec_loop_decode_call_ms: 449002.3644 ms
transfer:entropy_dec_loop_values_to_tensor_ms: 527616.1310 ms
entropy_dec_dequantize_ms: 540.8108 ms
synthesis_g_s_ms         : 9605.5118 ms
total_decompress_ms      : 1168116.0237 ms

--- 🚀 GPU 가속 시나리오 계산 (32x 병렬화) ---
원본 총 시간: 1168116.0237 ms

[절감 내역]
  데이터 전송 오버헤드 제거: 707762.7584 ms
  CPU 디코딩 시간 (원본): 449002.3644 ms
  GPU 디코딩 시간 (32x 병렬): 14031.3239 ms
  -> 디코딩 병렬화로 인한 절감: 434971.0405 ms
-------------------------------------------------
총 절감 시간 (오버헤드 + 병렬화): 1142733.7989 ms
✅ 최종 예상 시간: 25382.2247 ms
✅ 시간 절감률: 97.83 %


In [4]:
25382.2247 / 1000

25.3822247